# O Problema
Sliding Puzzle - Bloco Deslizante

In [ ]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

# Resolver o quebra-cabeças usando Buscas

In [ ]:
from copy import deepcopy
import numpy as np
from itertools import combinations

class Estado:
    def __init__(self, array):
        self.array = np.array(array)
    
    def __repr__(self):
        return str(self.array[0]) + '\n' + str(self.array[1]) + '\n' + str(self.array[2]) + '\n'
    
    def __eq__(self, other):
        return (self.array == other.array).all()


class FilaPrioridade:
    def __init__(self):
        self.list = []

    def put(self, item, priority):
        self.list.append((priority, item))
        self.list.sort(key=lambda x: x[0])

    def get(self):
        return self.list.pop(0)[1]
    
    def EspacoVazio(self):
        return len(self.list) == 0

class No:
    def __init__(self, estadoInicial):
        self.estado = estadoInicial
        self.pos = self.getPos()
        self.caminho = [estadoInicial]
    
    def AlcancaNovoEstado(self, novoEstado):
        novoNo = deepcopy(self)
        novoNo.estado = novoEstado
        novoNo.pos = novoNo.getPos()
        novoNo.caminho.append(novoEstado)
        
        return novoNo
    
    def getPos(self):
        for i in range(3):
            for j in range(3):
                if self.estado.array[i][j] == 0:
                    return i, j

def VerificaPosicao(no, estadoParada):
    return no.estado == estadoParada

In [ ]:
posicaoCorreta = [[1,2,3], [4,5,6], [7,8,0]]    # 0 para 'EspacoVazio'
posicaoCorreta = Estado(posicaoCorreta)
print(posicaoCorreta)

[1 2 3]
[4 5 6]
[7 8 0]



#A*
O algoritmo recebe:

*   O nó inicial
*   O nó final
*   Uma heurística

Começando pelo nó inicial, ele pega todos os vizinhos do nó atual e aplica a função de heurística. Essa função retorna um número que indica qual é a distância pro nó final. O vizinho que tiver o menor valor é o mais perto do nó final, então esse vizinho se torna o nó atual. O mesmo procedimento é repetido até que o nó atual seja o nó final.

Como o algoritmo determina o custo entre as arestas ligadas nos vértices do grafo é muito importante, e essa função é determinada pela heurística pois é ela que vai direcionar a busca pro caminho correto. Não existe uma "regra universal" de como criar a função de heurística, mas nesse caso eu utilizei as duas mais conhecidas para a solução desse problema "Hamming" e "Manhattan".




##Hamming
A distância de Hamming basicamente consideramos a quantidade de números fora da posição correta. Justamente pela definição da heurística, basicamente o que eu fiz foi percorrer as linhas i, e j que representa as colunas para aquela linha e fazer através de um condicional diferenciar a quantidade de números fora da posição final.

## Manhattan
A heurística de Manhattan, por outro lado, considera, para cada número fora de posição, quantos movimentos serão necessários para posicioná-lo corretamente. Então, além de considerar os números fora de posição, também calcula a quantidade de movimentos necessários para move-los para a posição correta somado da profundidade da árvore. Nesse caso também se considera como a melhor opção a situação que precisará da menor quantidade de movimentos possíveis para ser resolvida.


In [ ]:
def AEstrela(estadoInicial, estadoParada, heuristica):
    tipoHeuristica = ["Hamming", "Manhattan"]
    if heuristica not in tipoHeuristica:
        print("Opção inválida")
    elif heuristica == "Hamming":
        hn = lambda no: sum([sum([1 for i in range(3) for j in range(3) if no.estado.array[i][j] != estadoParada.array[i][j]])])
    elif heuristica == "Manhattan":
        hn = lambda no: sum([sum([abs(i-j) for i, j in zip(*no.estado.array.nonzero())])])



    no = No(estadoInicial)
    frg = FilaPrioridade()
    frg.put(no, 0)
    cnt = 0
    while True:
        cnt += 1
        
        if frg.EspacoVazio():
            return False
        no = frg.get()
        if VerificaPosicao(no, estadoParada):
            print("Solução encontrada em {} movimentos".format(cnt))    
            return no.caminho
        x, y = no.pos
        for i, j in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            if i+x < 0 or i+x >= 3 or j+y < 0 or j+y >= 3:
                continue
            novoEstado = deepcopy(no.estado)
            novoEstado.array[x+i][y+j], novoEstado.array[x][y] = novoEstado.array[x][y], novoEstado.array[x+i][y+j]
            novoNo = no.AlcancaNovoEstado(novoEstado)
            if novoNo.estado in no.caminho:        
                continue
            frg.put(novoNo, len(novoNo.caminho)-1 + hn(novoNo))  

In [16]:
tipoHeuristica = ["Hamming", "Manhattan"]
AEstrela(Estado([[1,2,3],[4,5,6],[7,0,8]]), posicaoCorreta, tipoHeuristica[1])
#TESTE MANHATTAN

Solução encontrada em 4 movimentos


[[1 2 3]
 [4 5 6]
 [7 0 8], [1 2 3]
 [4 5 6]
 [7 8 0]]

## Discorra sobre o desempenho do método em questões de:


1.   Consumo de memória
2.   Desempenho



---


1. Não cheguei a testar o consumo de memória nesse algoritmo, portanto a minha conclusão para esse tópico é baseada em suposições. Acredito que o consumo de memória nesse algoritmo seja intenso, pois a gente poderia ter um cenário onde árvore de estados pode crescer indefinidamente sem haver nenhuma poda. Outro problema pra esse consumo de memória seria que no problema do quebra cabeça de 8 peças nós temos cenários onde é impossível chegar no estado final, fazendo com que tenhamos infinitas posssibilidades de estados, logo o critério de parada se torna impossível de ser atingido, fazendo com que o algoritmo entre em loop nesses casos.

2. Considerando o tópico anterior, onde vimos o mesmo problema sendo resolvido,
utilizando buscas sem informação (largura e profundidade), o uso de heurísticas permitiu um desempenho muito melhor em termos de tempo sendo que o A* Hamming em todos os cenários apresentou melhor desempenho. Enquanto que na busca em largura nós tinhamos um tempo de execução muito superior, chegando a quase 40 minutos nos mesmos 3 cenários testados para o nosso estado final, a busca com informação, utilizando o algoritmo A* utilizando uma heurística foi até 90x mais rápido, e em muitas vezes consegue chegar a resposta ótima, sendo que na busca em largura sempre temos a solução ótima. 





#TESTES
ESTADO FINAL

 1, 2, 3,

 4, 5, 6,

 7, 8, 0

ESTADOS INICIAIS

 1, 2, 5,

 3, 0, 4,

 6, 7, 8


---

 1, 2, 0,

 3, 4, 5

 6, 7, 8



---

 3, 2, 0

 6, 1, 5

 7, 4, 8


---
JOGO 1

A* (Hamming): 59 segundos de execução, 22746 movimentos.

A* (Manhattan): Parei a execução após 10 minutos

JOGO 2

A* (Hamming): 39 segundos de execução, 17466 movimentos.

A* (Manhattan): Parei a execução após 10 minutos

JOGO 3

A* (Hamming): 38 segundos de execução 17394 movimentos

A* (Manhattan): Parei a execução após 30 minutos

In [12]:
tipoHeuristica = ["Hamming", "Manhattan"]
AEstrela(Estado([[1,2,5],[3,0,4],[6,7,8]]), posicaoCorreta, tipoHeuristica[0])
#JOGO 1 Heuristica Hamming

Solução encontrada em 22746 movimentos


[[1 2 5]
 [3 0 4]
 [6 7 8], [1 2 5]
 [3 4 0]
 [6 7 8], [1 2 5]
 [3 4 8]
 [6 7 0], [1 2 5]
 [3 4 8]
 [6 0 7], [1 2 5]
 [3 0 8]
 [6 4 7], [1 2 5]
 [0 3 8]
 [6 4 7], [1 2 5]
 [6 3 8]
 [0 4 7], [1 2 5]
 [6 3 8]
 [4 0 7], [1 2 5]
 [6 3 8]
 [4 7 0], [1 2 5]
 [6 3 0]
 [4 7 8], [1 2 5]
 [6 0 3]
 [4 7 8], [1 2 5]
 [0 6 3]
 [4 7 8], [0 2 5]
 [1 6 3]
 [4 7 8], [2 0 5]
 [1 6 3]
 [4 7 8], [2 5 0]
 [1 6 3]
 [4 7 8], [2 5 3]
 [1 6 0]
 [4 7 8], [2 5 3]
 [1 0 6]
 [4 7 8], [2 0 3]
 [1 5 6]
 [4 7 8], [0 2 3]
 [1 5 6]
 [4 7 8], [1 2 3]
 [0 5 6]
 [4 7 8], [1 2 3]
 [4 5 6]
 [0 7 8], [1 2 3]
 [4 5 6]
 [7 0 8], [1 2 3]
 [4 5 6]
 [7 8 0]]

In [ ]:
tipoHeuristica = ["Hamming", "Manhattan"]
AEstrela(Estado([[1,2,5],[3,0,4],[6,7,8]]), posicaoCorreta, tipoHeuristica[1])
#JOGO 1 Heuristica Manhattan

In [11]:
tipoHeuristica = ["Hamming", "Manhattan"]
AEstrela(Estado([[1,2,0],[3,4,5],[6,7,8]]), posicaoCorreta, tipoHeuristica[0])
#JOGO 2 Heuristica Hamming

Solução encontrada em 17466 movimentos


[[1 2 0]
 [3 4 5]
 [6 7 8], [1 0 2]
 [3 4 5]
 [6 7 8], [1 4 2]
 [3 0 5]
 [6 7 8], [1 4 2]
 [0 3 5]
 [6 7 8], [1 4 2]
 [6 3 5]
 [0 7 8], [1 4 2]
 [6 3 5]
 [7 0 8], [1 4 2]
 [6 3 5]
 [7 8 0], [1 4 2]
 [6 3 0]
 [7 8 5], [1 4 2]
 [6 0 3]
 [7 8 5], [1 4 2]
 [0 6 3]
 [7 8 5], [1 4 2]
 [7 6 3]
 [0 8 5], [1 4 2]
 [7 6 3]
 [8 0 5], [1 4 2]
 [7 0 3]
 [8 6 5], [1 0 2]
 [7 4 3]
 [8 6 5], [1 2 0]
 [7 4 3]
 [8 6 5], [1 2 3]
 [7 4 0]
 [8 6 5], [1 2 3]
 [7 4 5]
 [8 6 0], [1 2 3]
 [7 4 5]
 [8 0 6], [1 2 3]
 [7 4 5]
 [0 8 6], [1 2 3]
 [0 4 5]
 [7 8 6], [1 2 3]
 [4 0 5]
 [7 8 6], [1 2 3]
 [4 5 0]
 [7 8 6], [1 2 3]
 [4 5 6]
 [7 8 0]]

In [ ]:
tipoHeuristica = ["Hamming", "Manhattan"]
AEstrela(Estado([[1,2,0],[3,4,5],[6,7,8]]), posicaoCorreta, tipoHeuristica[1])
#JOGO 2 Heuristica Manhattan

In [13]:
tipoHeuristica = ["Hamming", "Manhattan"]
AEstrela(Estado([[3,2,0],[6,1,5],[7,4,8]]), posicaoCorreta, tipoHeuristica[0])
#JOGO 3 Heuristica Hamming

Solução encontrada em 17394 movimentos


[[3 2 0]
 [6 1 5]
 [7 4 8], [3 0 2]
 [6 1 5]
 [7 4 8], [3 1 2]
 [6 0 5]
 [7 4 8], [3 1 2]
 [0 6 5]
 [7 4 8], [0 1 2]
 [3 6 5]
 [7 4 8], [1 0 2]
 [3 6 5]
 [7 4 8], [1 2 0]
 [3 6 5]
 [7 4 8], [1 2 5]
 [3 6 0]
 [7 4 8], [1 2 5]
 [3 0 6]
 [7 4 8], [1 2 5]
 [0 3 6]
 [7 4 8], [0 2 5]
 [1 3 6]
 [7 4 8], [2 0 5]
 [1 3 6]
 [7 4 8], [2 3 5]
 [1 0 6]
 [7 4 8], [2 3 5]
 [1 4 6]
 [7 0 8], [2 3 5]
 [1 4 6]
 [7 8 0], [2 3 5]
 [1 4 0]
 [7 8 6], [2 3 0]
 [1 4 5]
 [7 8 6], [2 0 3]
 [1 4 5]
 [7 8 6], [0 2 3]
 [1 4 5]
 [7 8 6], [1 2 3]
 [0 4 5]
 [7 8 6], [1 2 3]
 [4 0 5]
 [7 8 6], [1 2 3]
 [4 5 0]
 [7 8 6], [1 2 3]
 [4 5 6]
 [7 8 0]]

In [ ]:
tipoHeuristica = ["Hamming", "Manhattan"]
AEstrela(Estado([[3,2,0],[6,1,5],[7,4,8]]), posicaoCorreta, tipoHeuristica[1])
#JOGO 3 Heuristica Manhattan